<a href="https://colab.research.google.com/github/Nateme16/geo-aquawatch-water-quality/blob/main/MDNtest/tryMDN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap --quiet
import ee
ee.Authenticate()
ee.Initialize()



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=gHF_PGdOkHCox3fAk1ODXwlazw0jtywBndZq9OOpmnE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh14NTpLu6_1L9jvL-bGkrC_6yqF6VeDUqQgzoAyH3yBATcH7L7ewU

Successfully saved authorization token.


In [ ]:

#get rid of clouds and keep just the water
#old s2 cloudmask and water, there is a different new one, but keeping it simple
def cloudmask(img):
                  mask = img.select(['SCL']).eq(6)
                  notcloud=img.select(['QA60']).eq(0)
                  img=img.updateMask(mask)
                  img=img.updateMask(notcloud)
                  return img

#get same image as Csaba
image= cloudmask(ee.Image('COPERNICUS/S2_SR/20200116T155609_20200116T155605_T18SUJ')) 

input1= image.select(["B4","B3","B2","B6","B1"]).multiply(.0001)




In [ ]:
#need to transform somewhere here?

array_image = input1.float().toArray() 

# Point to the model hosted on AI Platform.  If you specified a region other
# than the default (us-central1) at model creation, specify it here.

model = ee.Model.fromAiPlatformPredictor(
    projectName='ne-recreation-1545342727047',
    modelName='MDN',
    version='MDN',
    # Can be anything, but don't make it too big.
    inputTileSize=[8, 8],
    # Keep this the same as your training data.
    proj=ee.Projection('EPSG:4326').atScale(30),
    fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)


predictions = model.predictImage(array_image)
tss = predictions.arrayFlatten([['tss']])

#print(tss)

#bandNames = predictions.bandNames();
#print('Band names:', bandNames)

Map.addLayer(tss,[],"predict")



In [ ]:
import geemap
Map = geemap.Map(center=[39.2226,-76.3007], zoom= 9)
Map

Point (-76.2599, 39.2461) at 38m/px
AI Platform prediction service responded with error code 400: '{
    "error": "{{function_node __inference_signature_wrapper_15103_specialized_for_StatefulPartitionedCall_at_tf_graph}} {{function_node __inference_signature_wrapper_15103_specialized_for_StatefulPartitionedCall_at_tf_graph}} In[0] and In[1] has different ndims: [1,8,8,5] vs. [5,100]\n\t [[{{function_node __inference__wrapped_model_13972}}{{node MDN/dense_54/Relu}}]]\n\t [[StatefulPartitionedCall]]"
}'.
